In [1]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00


In [2]:
from transformers import VisionEncoderDecoderModel, ViltFeatureExtractor, AutoTokenizer
import torch
from PIL import Image

In [3]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

Some weights of the model checkpoint at nlpconnect/vit-gpt2-image-captioning were not used when initializing VisionEncoderDecoderModel: ['decoder.transformer.h.11.attn.bias', 'decoder.transformer.h.1.attn.bias', 'decoder.transformer.h.2.attn.masked_bias', 'decoder.transformer.h.10.attn.masked_bias', 'decoder.transformer.h.3.attn.bias', 'decoder.transformer.h.0.attn.bias', 'decoder.transformer.h.4.attn.bias', 'decoder.transformer.h.7.crossattention.bias', 'decoder.transformer.h.0.crossattention.bias', 'decoder.transformer.h.11.crossattention.masked_bias', 'decoder.transformer.h.0.crossattention.masked_bias', 'decoder.transformer.h.8.crossattention.bias', 'decoder.transformer.h.2.attn.bias', 'decoder.transformer.h.5.crossattention.masked_bias', 'decoder.transformer.h.4.attn.masked_bias', 'decoder.transformer.h.11.crossattention.bias', 'decoder.transformer.h.5.attn.masked_bias', 'decoder.transformer.h.7.attn.bias', 'decoder.transformer.h.11.attn.masked_bias', 'decoder.transformer.h.5.cros

In [6]:
from transformers.models.vit.feature_extraction_vit import ViTFeatureExtractor
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [7]:
device = torch.device ("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_featur

In [8]:
max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}


In [9]:
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [10]:
predict_step(['licensed-image.jfif'])

['a man in a suit and tie posing for a picture']

In [12]:
predict_step(['image.jfif'])

['a woman taking a picture of herself in a field']

In [13]:
predict_step(['awesome-solitude-bess-hamiti.jpg'])

['a tree in the middle of a field with a sky background']